In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
import glob, re, os
import subprocess


import scipy.linalg as lng

from smt.applications import EGO
from smt.surrogate_models import KRG
from smt.sampling_methods import LHS
from smt.applications import EGO
from smt.surrogate_models import KRG, XSpecs


import scienceplots
plt.style.use('default')
plt.style.use(['science', 'high-vis', 'grid'])

In [ ]:
# PATH_BUILD_DIR = "/work/utox/users/helleboid_work_utox/OptiDopi/build/"
# PATH_COST_CPP = "/work/utox/users/helleboid_work_utox/OptiDopi/build/apps/spad_cost_function"

PATH_BUILD_DIR = "/home/remi/DEV/OptiDopi/build/"
PATH_COST_CPP = "/home/remi/DEV/OptiDopi/build/apps/spad_cost_function"
RES_FILE = "RES/res_cost_func.csv"
os.makedirs("EGO_Results/", exist_ok=True)
# Recompile the code first.
#subprocess.run(["make", f"-j 32"], cwd=PATH_BUILD_DIR)

In [ ]:
def function_test_1d(doping_acceptor):
    doping_donor = 20.0
    length_intrinsic = 0.0
    subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{doping_donor}",f"{doping_acceptor}", RES_FILE])
    length_intrinsic, doping_donor, doping_acceptor, total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
    print(f"Cost function: {total_cost}")
    return total_cost

def function_test_2d(PARAMS):
    #print(PARAMS)
    Y = []
    for idx in range(len(PARAMS)):
        length_intrinsic = PARAMS[idx,0]
        #print(f'{length_intrinsic=}')
        log_doping_acceptor =  PARAMS[idx,1]
        subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{log_doping_acceptor}", RES_FILE])
        length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
        Y.append(total_cost)
        print(f"Total cost: {total_cost}")
    return np.array(Y)

vfunc1d = np.vectorize(function_test_1d)

vfunc2d = np.vectorize(function_test_2d)


In [ ]:
min_length_intrinsic = 0.0
max_length_intrinsic = 0.0
min_log_acceptor = 16.5
max_log_acceptor = 17.5


n_iter = 100
xlimits = np.array([[min_log_acceptor, max_log_acceptor]])
xspecs = XSpecs(xlimits=xlimits)
xdoe = np.atleast_2d(np.random.uniform(min_log_acceptor, max_log_acceptor, 2)).T
n_doe = xdoe.size

criterion = "EI"  #'EI' or 'SBO' or 'LCB'*

In [ ]:
ego = EGO(
    n_iter=n_iter,
    criterion=criterion,
    xdoe=xdoe,
    surrogate=KRG(xspecs=xspecs, print_global=False),
)

x_opt, y_opt, _, x_data, y_data = ego.optimize(fun=vfunc1d)
print("Minimum in x={:.1f} with f(x)={:.1f}".format(float(x_opt), float(y_opt)))

In [ ]:
x_line = np.linspace(min_log_acceptor, max_log_acceptor, 1000)
y_line = vfunc1d(x_line)

In [ ]:
x_min, y = x_line[np.argmin(y_line)], np.min(y_line)
print("Minimum in x={:.1f} with f(x)={:.1f}".format(float(x_min), float(y)))

In [ ]:
min_y_line = np.min(y_line)
min_y_data = np.min(y_data)
min_y_opt = np.min(y_opt)

min_global = np.min([min_y_line, min_y_data, min_y_opt])

y_line = y_line - min_global + 1e-3
y_data = y_data - min_global + 1e-3
y_opt = y_opt - min_global + 1e-3

print(f"y_opt: {y_data}")
fig, axs = plt.subplots(1, 1)
axs.plot(x_line, y_line, ".-", label="Cost function")
axs.plot(x_data, y_data, "o", label="Data")
axs.plot(x_opt, y_opt, "D", label="Optimum", c="y")

axs.set_xlabel("log(Acceptor)")
axs.set_ylabel("Cost function")
axs.set_yscale("log")